## **Q001**
## **While ingesting customer data from an external source, you notice duplicate entries. How would you remove duplicates and retain only the latest entry based on a timestamp column?**

## First lets try in SQL

In [0]:
%sql
CREATE TABLE sales_data (
    product_id STRING,
    date DATE,
    sales INT
);

INSERT INTO sales_data (product_id, date, sales) VALUES
('101', '2023-12-01', 100),
('101', '2023-12-02', 150),
('102', '2023-12-01', 200),
('102', '2023-12-02', 250);

SELECT * FROM sales_data;

product_id,date,sales
101,2023-12-01,100
101,2023-12-02,150
102,2023-12-01,200
102,2023-12-02,250


In [0]:
%sql

WITH CTE AS (
SELECT *,
row_number() OVER(PARTITION BY product_id ORDER BY date DESC) as rn
FROM sales_data)

SELECT product_id, date, sales FROM CTE WHERE rn=1;

product_id,date,sales
101,2023-12-02,150
102,2023-12-02,250


## Pyspark Solution

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [0]:
data = [("101", "2023-12-01", 100), ("101", "2023-12-02", 150), 
        ("102", "2023-12-01", 200), ("102", "2023-12-02", 250)]
columns = ["product_id","date","sales"]

df = spark.createDataFrame(data,columns)
df.display()

product_id,date,sales
101,2023-12-01,100
101,2023-12-02,150
102,2023-12-01,200
102,2023-12-02,250


In [0]:
## Casting date column from string to date format

df = df.withColumn('date',col('date').cast(DateType()))
# df = df.withColumn("date", to_date(col("date"),"yyyy-MM-dd")) -- Both would work


## Drop the duplicates
df = df.orderBy("product_id","date",ascending=[1,0]).dropDuplicates(subset=['product_id'])
df.display()

product_id,date,sales
101,2023-12-02,150
102,2023-12-02,250
